# Electricity demand forecasting

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from meteostat import Hourly

In [2]:
def load_power_plant_data(path: str = 'data/power_plant.csv') -> pd.DataFrame:
    df_power_plant = pd.read_csv(path, delimiter=';')
    df_power_plant = df_power_plant[df_power_plant['Power'] != '-']
    df_power_plant['Power'] = df_power_plant['Power'].str.replace(',', '.').astype(float)
    df_power_plant['Datetime'] = pd.to_datetime(df_power_plant['Datetime'], format='%d.%m.%Y %H:%M')
    df_power_plant = df_power_plant.reindex(columns=['Datetime', 'Power'])
    return df_power_plant

df = load_power_plant_data()

In [3]:
def fetch_holiday_data(years: list[int], region: str = 'de-be') -> pd.DataFrame:
    holiday_dates = []
    for year in years:
        url = f"https://digidates.de/api/v1/germanpublicholidays?year={year}&region={region}"
        response = requests.get(url)
        holidays = response.json()
        [holiday_dates.append(holiday) for holiday in holidays.keys()]

    df_holidays = pd.DataFrame(data={"Holiday": holiday_dates})
    return df_holidays

# years = df['Datetime'].dt.strftime("%Y").unique()
# df_holidays = fetch_holiday_data(years=years)
# df_holidays.to_csv('data/holidays.csv')

df_holidays = pd.read_csv('data/holidays.csv')
df_holidays['Holiday'] = pd.to_datetime(df_holidays['Holiday'], format='%Y-%m-%d').dt.date

In [4]:
def fetch_weather_data(start: pd.Timestamp, end: pd.Timestamp, station_id: str) -> pd.DataFrame:
    weather = Hourly(station_id, start, end)
    df_weather = weather.fetch()
    df_weather = df_weather.reset_index()

    df_weather = df_weather.rename(columns={
        'time': 'Datetime',
        'temp': 'Temperature',
        'dwpt': 'Dew Point',
        'rhum': 'Relative Humidity',
        'prcp': 'Precipitation',
        'snow': 'Snow Depth',
        'wdir': 'Wind Direction',
        'wspd': 'Average Wind Speed',
        'wpgt': 'Peak Wind Speed',
        'pres': 'Average Sea-Level Air Pressure',
        'tsun': 'Sunshine Duration',
        'coco': 'Weather Condition Code'
    })

    df_weather['Datetime'] = pd.to_datetime(df_weather['Datetime'])
    df_weather = df_weather.set_index('Datetime').sort_index()
    df_weather = df_weather.resample('15min').interpolate(method='linear')
    df_weather = df_weather.reset_index()
    return df_weather

# start = df['Datetime'].min()
# end = df['Datetime'].max()
# df_weather = fetch_weather_data(start=start, end=end, station_id='10582')
# df_weather.to_csv('data/weather.csv', index=False)

df_weather = pd.read_csv('data/weather.csv')
df_weather['Datetime'] = pd.to_datetime(df_weather['Datetime'])

In [5]:
# time-based features
df['Holiday'] = df['Datetime'].dt.date.isin(df_holidays['Holiday'])
df['Hour'] = df['Datetime'].dt.hour
df['DayOfWeek'] = df['Datetime'].dt.dayofweek
df['Month'] = df['Datetime'].dt.month
df['IsWeekend'] = df['DayOfWeek'].isin([5,6]).astype(int)

In [ ]:
# save dataset
df = pd.merge(df, df_weather, on=['Datetime'], how='left')
# df.to_csv('data/dataset.csv', sep=';', index=False)

### Day-ahead market price dataset

In [10]:
def load_market_data(path: str = 'data/day_ahead_prices.csv') -> pd.DataFrame:
    df_market = pd.read_csv(path, delimiter=';')
    df_market['Datetime'] = pd.to_datetime(df_market['Datetime'], format='%Y-%m-%d %H:%M:00')
    return df_market

df_market = load_market_data()

In [ ]:
df_market = df_market.sort_values('Datetime')
df_market['delta'] = df_market['Datetime'].diff()
print(df_market['delta'].value_counts().head(20))

delta
0 days 00:15:00    350253
0 days 00:00:00        40
0 days 01:15:00        10
Name: count, dtype: int64


In [ ]:
gaps = df_market[df_market['delta'] > pd.Timedelta('15min')]
for i, row in gaps.iterrows():
    start = df_market.loc[i - 1, 'Datetime']
    end = row['Datetime']
    print(f"Lücke zwischen {start} und {end} — Dauer: {row['delta']}")

Lücke zwischen 2015-03-29 01:45:00 und 2015-03-29 03:00:00 — Dauer: 0 days 01:15:00
Lücke zwischen 2016-03-27 01:45:00 und 2016-03-27 03:00:00 — Dauer: 0 days 01:15:00
Lücke zwischen 2017-03-26 01:45:00 und 2017-03-26 03:00:00 — Dauer: 0 days 01:15:00
Lücke zwischen 2018-03-25 01:45:00 und 2018-03-25 03:00:00 — Dauer: 0 days 01:15:00
Lücke zwischen 2019-03-31 01:45:00 und 2019-03-31 03:00:00 — Dauer: 0 days 01:15:00
Lücke zwischen 2020-03-29 01:45:00 und 2020-03-29 03:00:00 — Dauer: 0 days 01:15:00
Lücke zwischen 2021-03-28 01:45:00 und 2021-03-28 03:00:00 — Dauer: 0 days 01:15:00
Lücke zwischen 2022-03-27 01:45:00 und 2022-03-27 03:00:00 — Dauer: 0 days 01:15:00
Lücke zwischen 2023-03-26 01:45:00 und 2023-03-26 03:00:00 — Dauer: 0 days 01:15:00
Lücke zwischen 2024-03-31 01:45:00 und 2024-03-31 03:00:00 — Dauer: 0 days 01:15:00


In [ ]:
gaps = df_market[df_market['delta'] == pd.Timedelta('0min')]
for i, row in gaps.iterrows():
    start = df_market.loc[i - 1, 'Datetime']
    end = row['Datetime']
    print(f"Lücke zwischen {start} und {end} — Dauer: {row['delta']}")

Lücke zwischen 2015-10-25 02:45:00 und 2015-10-25 02:00:00 — Dauer: 0 days 00:00:00
Lücke zwischen 2015-10-25 02:00:00 und 2015-10-25 02:15:00 — Dauer: 0 days 00:00:00
Lücke zwischen 2015-10-25 02:15:00 und 2015-10-25 02:30:00 — Dauer: 0 days 00:00:00
Lücke zwischen 2015-10-25 02:30:00 und 2015-10-25 02:45:00 — Dauer: 0 days 00:00:00
Lücke zwischen 2016-10-30 02:45:00 und 2016-10-30 02:00:00 — Dauer: 0 days 00:00:00
Lücke zwischen 2016-10-30 02:00:00 und 2016-10-30 02:15:00 — Dauer: 0 days 00:00:00
Lücke zwischen 2016-10-30 02:15:00 und 2016-10-30 02:30:00 — Dauer: 0 days 00:00:00
Lücke zwischen 2016-10-30 02:30:00 und 2016-10-30 02:45:00 — Dauer: 0 days 00:00:00
Lücke zwischen 2017-10-29 02:45:00 und 2017-10-29 02:00:00 — Dauer: 0 days 00:00:00
Lücke zwischen 2017-10-29 02:00:00 und 2017-10-29 02:15:00 — Dauer: 0 days 00:00:00
Lücke zwischen 2017-10-29 02:15:00 und 2017-10-29 02:30:00 — Dauer: 0 days 00:00:00
Lücke zwischen 2017-10-29 02:30:00 und 2017-10-29 02:45:00 — Dauer: 0 days 0